## Calculate NGAM/GAM in metabolic model

In [1]:
import cobra

# open the metabolic model
s12 = cobra.io.read_sbml_model(os.getcwd()+'\\input\\iSH1474.xml')
model = s12.copy()

#Call biomass equation
biomass_eq = model.reactions.get_by_id('BIOMASS_S12_WT')

In [2]:
# NGAM calculation

# Following values were calculated using experimental values 
# Details are in the "Calculation of GAM and NGAM" section in the reference paper 
# (Han et al. Genome-scale metabolic network model and phenome of solvent-tolerant Pseudomonas putida S12)
# q= growth_rate/Y_G + maintenance coefficient(mc)

inverse_Y_G = 10  # 1/Y_G
mc = 0.077

import numpy as np

gam_am = {'h2o_c','atp_c','adp_c','h_c','pi_c'}

for m in gam_am:
    globals()[m] = model.metabolites.get_by_id(m)
    globals()[m+'_coef'] = biomass_eq.get_coefficient(globals()[m])

with model:
    model.reactions.get_by_id('ATPM').bounds = (-99999,99999)
    
    # Generate biomass equation without GAM and GAM-dependent components 
    biomass_eq.add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})
    
    # set the NGAM as objective function
    model.objective = 'ATPM'
    model.reactions.get_by_id("EX_glc__D_e").lower_bound = -mc # glucose uptake rate = 0.077
    biomass_eq.lower_bound = 0
    ngam = round(model.optimize().objective_value,2)
        
print('NGAM:',ngam)

NGAM: 1.67


In [3]:
def nearest_value(simulated_data, inverse_Y_G):
    
    tmp = 0
    gam = 0
    
    for i in simulated_data.items():
        if abs(inverse_Y_G-tmp) > abs(inverse_Y_G-i[1]):
            gam = i[0]
            tmp = i[1]
    
    return gam

In [4]:
# GAM calculation

# Set calibration values involved in the generation of other biomass compositions, such as dna, rna, etc
atp_rna = 0.17064788767095312
pi_inorg = 0.0035566392840948907
h2o_aa = 5.3105

model.reactions.get_by_id('ATPM').bounds = (ngam,ngam) # fix the ngam

gam_am = {'h2o_c','atp_c','adp_c','h_c','pi_c'}

glucose = [1,2,3,4,5] # mmol

simulated_data = {}

for coef in (np.arange(42.30,42.35,0.01)): # possible GAM values ('adp_c') 
    objective_values = []
    
    with model:
        for glc in glucose:
            model.reactions.get_by_id("EX_glc__D_e").lower_bound = -glc
            
            for m in gam_am:
                globals()[m+'_coef'] = biomass_eq.get_coefficient(globals()[m])
            
            # Generate biomass equation without GAM and GAM-dependent components 
            biomass_eq.add_metabolites({'h2o_c':-h2o_c_coef,'atp_c':-atp_c_coef,'adp_c':-adp_c_coef,'h_c':-h_c_coef,'pi_c':-pi_c_coef})
            
            # Generate biomass equation with GAM and GAM-dependent components by changing GAM value
            biomass_eq.add_metabolites({'h2o_c':round(-(coef-h2o_aa),6),'atp_c':round(-(coef+atp_rna),6),'adp_c':round(coef,6),'h_c':round(coef,6),'pi_c':round(coef-pi_inorg,6)})

            objective_values.append(model.optimize().objective_value)
            
        slope, intercept = np.polyfit(objective_values, glucose,1)
        simulated_data[coef]=slope
        print(coef, slope)

        
# get the value which most nearest with inverse_Y_G

gam = round(nearest_value(simulated_data, inverse_Y_G),2)
print('GAM:',gam)

42.3 9.999639172413893
42.309999999999995 10.0000989425288
42.31999999999999 10.000558712643478
42.32999999999999 10.00101848275865
42.33999999999999 10.001478252873602
42.34999999999999 10.001938022988606
GAM: 42.31


In [6]:
# For generation of Figure S1 in the reference paper
import pandas as pd

glucose = [1,2,3,4,5,6,7,8,9,10,11]
result = []

print('iSH1474 growth simulation values\n')

for glc in glucose:
    with model:
        model.reactions.get_by_id("EX_glc__D_e").lower_bound = -glc
        result.append(model.slim_optimize())
        
display(pd.DataFrame({'glucose uptake rate (mmol/gDCW/h)':glucose,'iSH1474 growth simulation':result}))

iSH1474 growth simulation values



,glucose uptake rate (mmol/gDCW/h),iSH1474 growth simulation
0,1,0.092321
1,2,0.192320
2,3,0.292319
3,4,0.392318
4,5,0.492317
5,6,0.592316
6,7,0.692315
7,8,0.792314
8,9,0.892313
9,10,0.992312
